In [ ]:
!nvidia-smi

Sun Oct 24 17:44:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip -q install torchaudio
# !pip -q install datasets

import pandas as pd
import numpy as np
import os
from tqdm import tqdm_notebook as tqdm
from shutil import copyfile

import librosa
import librosa.display
import scipy.io.wavfile as wav

import IPython.display as ipd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

     |████████████████████████████████| 2.9 MB 5.3 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.4 MB/s eta 0:00:34tcmalloc: large alloc 1147494400 bytes == 0x5590bf3e2000 @  0x7ffa896dc615 0x559084e7a4cc 0x559084f5a47a 0x559084e7d2ed 0x559084f6ee1d 0x559084ef0e99 0x559084eeb9ee 0x559084e7ebda 0x559084ef0d00 0x559084eeb9ee 0x559084e7ebda 0x559084eed737 0x559084f6fc66 0x559084eecdaf 0x559084f6fc66 0x559084eecdaf 0x559084f6fc66 0x559084eecdaf 0x559084e7f039 0x559084ec2409 0x559084e7dc52 0x559084ef0c25 0x559084eeb9ee 0x559084e7ebda 0x559084eed737 0x559084eeb9ee 0x559084e7ebda 0x559084eec915 0x559084e7eafa 0x559084eecc0d 0x559084eeb9ee
     |████████████████████████████████| 881.9 MB 20 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu111 requires torch==1.9.0, but you have torch 1.10.0 which is incompatible.
torchtext 0.10.0 re

In [ ]:
import pandas as pd
from torch.utils.data import Dataset

import torch

In [ ]:
train_rc = pd.read_csv('/content/drive/MyDrive/Tesi/train_rc.csv')
test_rc = pd.read_csv('/content/drive/MyDrive/Tesi/test_rc.csv')

In [ ]:
!7z x /content/drive/MyDrive/Tesi/dataset_bandPass2.zip -o/content/


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/Tesi/                                      1 file, 569072075 bytes (543 MiB)

Extracting archive: /content/drive/MyDrive/Tesi/dataset_bandPass2.zip
 59% 4096 Open              --
Path = /content/drive/MyDrive/Tesi/dataset_bandPass2.zip
Type = zip
Physical Size = 569072075

  0%      1% 113 - dataset_bandPass2/186_2b4_Al_mc_AKGC417L_0_bandPass2.wav                                                                     2% 165 - dataset_bandPass2/114_1b4_Lr_mc_AKGC417L_2_bandPass2.wav

In [ ]:
print(train_rc.shape)
print(test_rc.shape)

(5518, 2)
(1380, 2)


In [ ]:
print(train_rc['label'].unique())

['none' 'wheeze' 'crackle' 'both']


In [ ]:
train_4 = train_rc.copy()
train_4 = train_4.replace('none', 0)
train_4 = train_4.replace('crackle', 1)
train_4 = train_4.replace('wheeze', 2)
train_4 = train_4.replace('both',3)
train_4['filename']="/content/dataset_bandPass2/"+train_4['filename']+'_bandPass2.wav'
train_4

,filename,label
0,/content/dataset_bandPass2/194_1b1_Pr_sc_Medit...,0
1,/content/dataset_bandPass2/113_1b1_Ar_sc_Litt3...,0
2,/content/dataset_bandPass2/211_1p2_Pr_mc_AKGC4...,0
3,/content/dataset_bandPass2/120_1b1_Lr_sc_Medit...,0
4,/content/dataset_bandPass2/203_1p4_Pr_mc_AKGC4...,0
...,...,...
5513,/content/dataset_bandPass2/158_1p4_Tc_mc_AKGC4...,0
5514,/content/dataset_bandPass2/154_1b3_Ll_mc_AKGC4...,1
5515,/content/dataset_bandPass2/113_1b1_Lr_sc_Litt3...,0
5516,/content/dataset_bandPass2/204_2b5_Al_mc_AKGC4...,2


In [ ]:
test_4 = test_rc.copy()
test_4 = test_4.replace('none', 0)
test_4 = test_4.replace('crackle', 1)
test_4 = test_4.replace('wheeze', 2)
test_4 = test_4.replace('both',3)
test_4['filename']="/content/dataset_bandPass2/"+test_4['filename']+'_bandPass2.wav'
test_4

,filename,label
0,/content/dataset_bandPass2/157_1b1_Pr_sc_Medit...,1
1,/content/dataset_bandPass2/130_2b2_Al_mc_AKGC4...,1
2,/content/dataset_bandPass2/210_1b1_Ar_sc_Medit...,0
3,/content/dataset_bandPass2/163_8b3_Ll_mc_AKGC4...,0
4,/content/dataset_bandPass2/178_1b6_Pl_mc_AKGC4...,0
...,...,...
1375,/content/dataset_bandPass2/147_1b2_Tc_mc_AKGC4...,0
1376,/content/dataset_bandPass2/192_2b2_Al_mc_LittC...,3
1377,/content/dataset_bandPass2/130_1p2_Pl_mc_AKGC4...,1
1378,/content/dataset_bandPass2/200_2p3_Pr_mc_AKGC4...,0


In [ ]:
train_4.groupby('label').count()

,filename
label,
0,2946
1,1469
2,704
3,399


In [ ]:
test_4.groupby('label').count()

,filename
label,
0,696
1,395
2,182
3,107


In [ ]:
!pip -q install torchaudio

In [ ]:
import torchaudio
duration = 4
freq = 16
class AudiosDataset(Dataset):
    def __init__(self, paths=None, targets=None, augment=False) -> None:
        self.paths = paths
        self.targets = targets
        self.augment = augment
        
    
    def __len__(self) -> int:
        return len(self.paths)

    def __getitem__(self, idx: int) -> dict:
        y, sr = torchaudio.load(self.paths.iloc[idx])#, normalization=True)
        """
        if self.augment:
            effects = [
                    ["lowpass", "-1", "300"], 
                    ["speed", f"{random.uniform(0.7, 1.3)}"],  # change speed
                  ]
            y, sr = torchaudio.sox_effects.apply_effects_tensor(
                y, sr, effects)
        #
            
        if self.augment:
            # augment sound in order to imitate the room change
            rir = self.rir[:, int(16000*1.1):int(16000*1.3)]
            rir = rir / torch.norm(rir, p=2)
            rir = torch.flip(rir, [1])
            y = torch.nn.functional.conv1d(y[None, ...], rir[None, ...])[0]
        
        """
        # here more augmentations could be used
        y = torchaudio.transforms.Resample(orig_freq=sr, new_freq=freq*1000)(y)

        """
        if self.augment:
            y = (0.5)*torch.randn(y.shape)
        """
        fixed_length = (freq*1000) * duration
        
        # returning result
        if y.shape[1] < fixed_length:
            y = torch.nn.functional.pad(
              y, (0, fixed_length - y.shape[1]))
        else:
            y = y[:, :fixed_length]
        result = {"target":self.targets.iloc[idx], "representation":y}
        return result

In [ ]:
train_ds = AudiosDataset(train_4['filename'], train_4['label'])
test_ds = AudiosDataset(test_4['filename'], test_4['label'])

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

batch_size = 8
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    )
}

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 5.5 MB/s 
     |████████████████████████████████| 56 kB 4.4 MB/s 
     |████████████████████████████████| 895 kB 37.2 MB/s 
     |████████████████████████████████| 596 kB 42.8 MB/s 
     |████████████████████████████████| 3.3 MB 44.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from torch import nn
import torch.nn.functional as F

class ClassificationHead(nn.Module):
    """Head for wav2vec2 classification task."""

    def __init__(self):
        super().__init__()
        self.dense = nn.Linear(768, 512)#config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(0.5)
        self.out_proj = nn.Linear(512, 4)#config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

class Classifier(nn.Module):
    def __init__(self, extractor):
        super().__init__()
        self.extractor = extractor
        self.classifier = ClassificationHead()
    def forward(self, x):
        features = self.extractor(torch.squeeze(x))
        features = features[0]
        res = torch.mean(features, dim=1)
        logit = self.classifier(res)
        return logit

In [ ]:
from tqdm import tqdm
import math
from torch import autograd
import datetime


def train(model, opt, scheduler, loss_fn, epochs, data_tr, data_val, lr, last_epoch=0, data_augmentation = False, max_stable=5):
    best_val_loss = 1e9
    counter = 0
    augmented = '_augmented' if data_augmentation else ''
    nan=False
    for epoch in range(last_epoch, epochs):
        correct_predictions = 0
        count=0
        #tic = time()
        print('* Epoch %d/%d' % (epoch+1, epochs))

        avg_loss = 0
        model.train()  # train mode
        progress = tqdm(data_tr)
        for batch in progress:
            loss = 0
            # data to device
            X_batch, Y_batch = batch["representation"], batch["target"]
            # print(X_batch.shape)
            X_batch = X_batch.to(DEVICE)
            Y_batch = Y_batch.to(DEVICE)
            # set parameter gradients to zero
            opt.zero_grad()
            # forward
            Y_pred = model(X_batch)
            _, preds = torch.max(Y_pred, dim=1)
            preds = preds.to(DEVICE)
            Y_pred = Y_pred.to(DEVICE)

            loss = loss_fn(Y_pred, Y_batch)# forward-pass
            nan = math.isnan(loss)

            if nan:
              count=count+1
              nan=False
              continue
            correct_predictions += torch.sum(preds == Y_batch)
            progress.set_description(f'Loss: {loss:.2f} #nan->{count}')
            
            loss.backward()  # backward-pass
            opt.step()  # update weights
            if not scheduler is None:
                scheduler.step()
            # calculate loss to show the user
            avg_loss += loss
        avg_loss /= (len(data_tr)-(batch_size*count))
        avg_acc = correct_predictions.double() / ((len(data_tr)-count)*batch_size)
        
        print('loss: %f' % avg_loss)
        print('acc: %f' % avg_acc)
        # show intermediate results


        model.eval()  # testing mode
        val_loss = 0
        val_acc = 0
        print("start validation")
        count_loss_val=0
        progress_val = tqdm(data_val)
        correct_predictions_val = 0
        for v_b in progress_val:
            X_val, Y_val = v_b["representation"], v_b["target"]
            # X_val = X_val.to(DEVICE)#
            # Y_val = Y_val.to(DEVICE)#
            Y_hat = model(X_val.to(DEVICE)).detach().cpu()# detach and put into cpu
            
            _, preds_val = torch.max(Y_hat, dim=1)
            # preds_val = preds_val.to(DEVICE)#

            loss_val = loss_fn(Y_hat, Y_val)
            nan_loss = math.isnan(loss_val) #check loss==nan
            if nan_loss:
              count_loss_val = count_loss_val+1
              nan_loss = False
              continue
            #se trova un nan, salta tutto il batch
            correct_predictions_val += torch.sum(preds_val == Y_val)
            progress_val.set_description(f'Loss: {loss_val:.2f} #nan->{count_loss_val}')
            val_loss += loss_val

        val_loss /= (len(data_val)-(batch_size*count_loss_val))
        val_acc = correct_predictions_val.double() / ((len(data_val)-count_loss_val)*batch_size)
        print( f"validation loss: {val_loss}")
        print( f"validation accuracy: {val_acc}")
        if val_loss <= best_val_loss and val_loss > 0:
            counter = 0
            now = datetime.datetime.now()
            print(f'Save new model!- {now.day}/{now.month} h{now.hour+2}:{now.minute}')
            best_val_loss = val_loss
            torch.save(model.state_dict(), f'/content/drive/MyDrive/Tesi/Wav2vec2/final_80-20_bandPass2_dropout=0.5_4-class_wav2vec2_lr={lr}_batch={batch_size}_{freq}mhz_{duration}sec{augmented}.bin')
        else:
            counter += 1
        print("--------------------")
        if counter == max_stable:
            break

----------------------------

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

freq=16
duration=4
train_ds = AudiosDataset(train_4['filename'], train_4['label'])
test_ds = AudiosDataset(test_4['filename'], test_4['label'])

batch_size = 8
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Modello finale

In [ ]:
from transformers import Wav2Vec2Model
DEVICE = 'cuda'
m = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")# torch.hub.load('s3prl/s3prl', 'audio_albert').to(DEVICE)

Downloading:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/360M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 4secondi tanh dropout=0.5 batch=8 wav2vec2 final

* Epoch 1/100


  0%|          | 0/689 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Loss: 1.14 #nan->0: 100%|██████████| 689/689 [11:33<00:00,  1.01s/it]


loss: 1.184016
acc: 0.516509
start validation


Loss: 1.03 #nan->0: 100%|██████████| 172/172 [01:07<00:00,  2.56it/s]


validation loss: 1.1479833126068115
validation accuracy: 0.5050872093023255
Save new model!- 24/10 h17:31
--------------------
* Epoch 2/100


...

Loss: 0.92 #nan->0: 100%|██████████| 689/689 [11:27<00:00,  1.00it/s]


loss: 0.985915
acc: 0.602866
start validation


Loss: 0.87 #nan->0: 100%|██████████| 172/172 [01:06<00:00,  2.57it/s]


validation loss: 0.9861192107200623
validation accuracy: 0.6191860465116279
Save new model!- 24/10 h18:22
--------------------
* Epoch 6/100


Loss: 0.76 #nan->0:  93%|█████████▎| 638/689 [10:39<00:49,  1.04it/s]

In [ ]:
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Wav2vec2/final_80-20_bandPass2_dropout=0.5_4-class_wav2vec2_lr=1e-05_batch=8_16mhz_4sec.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=5)
#16mhz 4secondi tanh dropout=0.5 batch=8 wav2vec2 final

* Epoch 6/100


  0%|          | 0/689 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Loss: 0.91 #nan->0: 100%|██████████| 689/689 [10:54<00:00,  1.05it/s]


loss: 0.908625
acc: 0.634253
start validation


Loss: 0.94 #nan->0: 100%|██████████| 172/172 [01:22<00:00,  2.10it/s]


validation loss: 0.91208416223526
validation accuracy: 0.6468023255813954
Save new model!- 24/10 h20:1
--------------------
* Epoch 7/100


...

validation loss: 0.8941221237182617
validation accuracy: 0.6954941860465116
--------------------
* Epoch 18/100


Loss: 0.39 #nan->0: 100%|██████████| 689/689 [10:58<00:00,  1.05it/s]


loss: 0.578419
acc: 0.783926
start validation


Loss: 1.06 #nan->0: 100%|██████████| 172/172 [01:18<00:00,  2.20it/s]


validation loss: 0.8394388556480408
validation accuracy: 0.7122093023255814
--------------------


### Tentativi precedenti

---------------------

In [ ]:
from transformers import Wav2Vec2Model
DEVICE = 'cuda'
m = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")# torch.hub.load('s3prl/s3prl', 'audio_albert').to(DEVICE)

Downloading:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/360M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 5secondi sigmoid dropout=0.7 batch=8 wav2vec2

* Epoch 1/100


  0%|          | 0/689 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Loss: 1.39 #nan->0: 100%|██████████| 689/689 [11:24<00:00,  1.01it/s]


loss: 1.184902
acc: 0.517054
start validation


Loss: 1.12 #nan->0: 100%|██████████| 172/172 [01:06<00:00,  2.58it/s]


validation loss: 1.1460881233215332
validation accuracy: 0.5174418604651163
Save new model!- 21/10 h21:0
--------------------
* Epoch 2/100


...

Loss: 1.28 #nan->0: 100%|██████████| 689/689 [11:20<00:00,  1.01it/s]


loss: 1.064666
acc: 0.551161
start validation


Loss: 0.92 #nan->0: 100%|██████████| 172/172 [01:05<00:00,  2.61it/s]


validation loss: 1.0620383024215698
validation accuracy: 0.5588662790697675
--------------------
* Epoch 8/100


Loss: 1.07 #nan->0:  16%|█▋        | 112/689 [01:54<09:41,  1.01s/it]

In [ ]:
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Wav2vec2/bandPass2_dropout=0.5_4-class_wav2vec2_lr=1e-05_batch=8_16mhz_4sec.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=6)
#16mhz 5secondi sigmoid dropout=0.7 batch=8 wav2vec2

* Epoch 7/100


  0%|          | 0/689 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Loss: 1.16 #nan->0: 100%|██████████| 689/689 [11:25<00:00,  1.01it/s]


loss: 1.058971
acc: 0.556604
start validation


Loss: 1.00 #nan->0: 100%|██████████| 172/172 [01:21<00:00,  2.10it/s]


validation loss: 1.0374774932861328
validation accuracy: 0.5813953488372093
Save new model!- 22/10 h11:4
--------------------
* Epoch 8/100


...

Loss: 0.18 #nan->0: 100%|██████████| 689/689 [11:28<00:00,  1.00it/s]


loss: 0.555743
acc: 0.787192
start validation


Loss: 1.64 #nan->0: 100%|██████████| 172/172 [01:22<00:00,  2.09it/s]


validation loss: 0.7825333476066589
validation accuracy: 0.7100290697674418
Save new model!- 22/10 h14:30
--------------------
* Epoch 24/100


Loss: 0.37 #nan->0:  24%|██▍       | 168/689 [02:53<08:44,  1.01s/it]

--------------------------

In [ ]:
from transformers import Wav2Vec2Model
DEVICE = 'cuda'
m = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")# torch.hub.load('s3prl/s3prl', 'audio_albert').to(DEVICE)

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-6
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 5secondi sigmoid dropout=0.7 batch=8 wav2vec2

* Epoch 1/100


  0%|          | 0/689 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Loss: 1.09 #nan->0: 100%|██████████| 689/689 [14:07<00:00,  1.23s/it]


loss: 1.213178
acc: 0.489115
start validation


Loss: 1.15 #nan->0: 100%|██████████| 172/172 [01:17<00:00,  2.21it/s]


validation loss: 1.1595408916473389
validation accuracy: 0.5174418604651163
Save new model!
--------------------
* Epoch 2/100


...

validation loss: 1.1611661911010742
validation accuracy: 0.5174418604651163
--------------------
* Epoch 6/100


Loss: 1.22 #nan->0: 100%|██████████| 689/689 [14:04<00:00,  1.23s/it]


loss: 1.172981
acc: 0.521771
start validation


Loss: 1.14 #nan->0: 100%|██████████| 172/172 [01:17<00:00,  2.23it/s]


validation loss: 1.1620391607284546
validation accuracy: 0.5174418604651163
--------------------


--------------------------

In [ ]:
DEVICE = 'cuda'
m = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")# torch.hub.load('s3prl/s3prl', 'audio_albert').to(DEVICE)

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


  0%|          | 0/689 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Loss: 1.31 #nan->0: 100%|██████████| 689/689 [13:07<00:00,  1.14s/it]


loss: 1.218668
acc: 0.483672
start validation


Loss: 1.09 #nan->0: 100%|██████████| 172/172 [01:11<00:00,  2.39it/s]


validation loss: 1.1638691425323486
validation accuracy: 0.5174418604651163
Save new model!
--------------------
* Epoch 2/100


...

Loss: 0.68 #nan->0: 100%|██████████| 689/689 [13:04<00:00,  1.14s/it]


loss: 0.758215
acc: 0.712264
start validation


Loss: 1.34 #nan->0: 100%|██████████| 172/172 [01:11<00:00,  2.39it/s]


validation loss: 0.850946307182312
validation accuracy: 0.654796511627907
--------------------
* Epoch 15/100


Loss: 0.44 #nan->0:  59%|█████▊    | 404/689 [07:42<05:23,  1.13s/it]

In [ ]:
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Wav2vec2/OK-4-class_wav2vec2_lr=1e-05_batch=8_16mhz_5sec.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=13)
#16mhz 5secondi tanh dropout=0.7 batch=8 mockingjay


* Epoch 14/100


  0%|          | 0/689 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Loss: 0.99 #nan->0: 100%|██████████| 689/689 [13:37<00:00,  1.19s/it]


loss: 0.766104
acc: 0.700835
start validation


Loss: 1.06 #nan->0: 100%|██████████| 172/172 [01:14<00:00,  2.31it/s]


validation loss: 0.8281728029251099
validation accuracy: 0.6838662790697675
Save new model!
--------------------
* Epoch 15/100


...

validation loss: 0.8595086932182312
validation accuracy: 0.6991279069767442
--------------------
* Epoch 19/100


Loss: 0.34 #nan->0: 100%|██████████| 689/689 [13:38<00:00,  1.19s/it]


loss: 0.626658
acc: 0.770319
start validation


Loss: 1.36 #nan->0: 100%|██████████| 172/172 [01:14<00:00,  2.31it/s]


validation loss: 0.8677477240562439
validation accuracy: 0.6875
--------------------


----------------------

In [ ]:
DEVICE = 'cuda'
m = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")# torch.hub.load('s3prl/s3prl', 'audio_albert').to(DEVICE)

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-6
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


  0%|          | 0/689 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Loss: 1.30 #nan->0: 100%|██████████| 689/689 [13:39<00:00,  1.19s/it]


loss: 1.333638
acc: 0.390602
start validation


Loss: 1.22 #nan->0: 100%|██████████| 172/172 [01:14<00:00,  2.31it/s]


validation loss: 1.2548834085464478
validation accuracy: 0.5174418604651163
Save new model!
--------------------
* Epoch 2/100


...

loss: 1.199461
acc: 0.510522
start validation


Loss: 1.10 #nan->0: 100%|██████████| 172/172 [01:14<00:00,  2.30it/s]


validation loss: 1.148156762123108
validation accuracy: 0.5174418604651163
Save new model!
--------------------
* Epoch 6/100


Loss: 1.07 #nan->0:   3%|▎         | 23/689 [00:29<14:21,  1.29s/it]


KeyboardInterrupt: ignored

-----------------------

In [ ]:
DEVICE = 'cuda'
m = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")# torch.hub.load('s3prl/s3prl', 'audio_albert').to(DEVICE)

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-4
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


  0%|          | 0/689 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Loss: 0.96 #nan->0: 100%|██████████| 689/689 [13:38<00:00,  1.19s/it]


loss: 1.179919
acc: 0.516147
start validation


Loss: 1.16 #nan->0: 100%|██████████| 172/172 [01:14<00:00,  2.31it/s]


validation loss: 1.187211513519287
validation accuracy: 0.5174418604651163
Save new model!
--------------------
* Epoch 2/100


...

loss: 1.157414
acc: 0.531023
start validation


Loss: 1.15 #nan->0: 100%|██████████| 172/172 [01:14<00:00,  2.31it/s]


validation loss: 1.1649737358093262
validation accuracy: 0.5174418604651163
--------------------
* Epoch 5/100


Loss: 1.68 #nan->0:   1%|          | 7/689 [00:11<18:56,  1.67s/it]


KeyboardInterrupt: ignored

------------------------

In [ ]:
DEVICE = 'cuda'
m = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")# torch.hub.load('s3prl/s3prl', 'audio_albert').to(DEVICE)

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =8e-6
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


  0%|          | 0/689 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Loss: 1.12 #nan->0: 100%|██████████| 689/689 [13:40<00:00,  1.19s/it]


loss: 1.223879
acc: 0.489478
start validation


Loss: 1.12 #nan->0: 100%|██████████| 172/172 [01:14<00:00,  2.31it/s]


validation loss: 1.1533385515213013
validation accuracy: 0.5174418604651163
Save new model!
--------------------
* Epoch 2/100


...

Loss: 1.31 #nan->0: 100%|██████████| 689/689 [13:40<00:00,  1.19s/it]


loss: 1.029122
acc: 0.570936
start validation


Loss: 1.04 #nan->0: 100%|██████████| 172/172 [01:14<00:00,  2.30it/s]


validation loss: 0.983681321144104
validation accuracy: 0.590843023255814
Save new model!
--------------------
* Epoch 9/100


Loss: 0.97 #nan->0:  35%|███▌      | 244/689 [04:51<08:50,  1.19s/it]

In [ ]:
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Wav2vec2/OK-4-class_wav2vec2_lr=8e-06_batch=8_16mhz_5sec.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =8e-6
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=8)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 9/100


  0%|          | 0/689 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Loss: 0.87 #nan->0: 100%|██████████| 689/689 [13:27<00:00,  1.17s/it]


loss: 0.990975
acc: 0.594158
start validation


Loss: 0.99 #nan->0: 100%|██████████| 172/172 [01:13<00:00,  2.35it/s]


validation loss: 0.9344909191131592
validation accuracy: 0.6351744186046512
Save new model!
--------------------
* Epoch 10/100


...

Loss: 0.66 #nan->0: 100%|██████████| 689/689 [13:27<00:00,  1.17s/it]


loss: 0.633279
acc: 0.769231
start validation


Loss: 1.24 #nan->0: 100%|██████████| 172/172 [01:13<00:00,  2.34it/s]


validation loss: 0.9445477724075317
validation accuracy: 0.6802325581395349
--------------------
* Epoch 23/100


Loss: 0.40 #nan->0:  50%|████▉     | 344/689 [06:45<06:44,  1.17s/it]

---------------------------------
Wav2vec2 con clip di 3 secondi

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

freq=16
duration=3
train_ds = AudiosDataset(train_4['filename'], train_4['label'])
test_ds = AudiosDataset(test_4['filename'], test_4['label'])

batch_size = 8
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
DEVICE = 'cuda'
m = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")# torch.hub.load('s3prl/s3prl', 'audio_albert').to(DEVICE)

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 3secondi tanh dropout=0.7 batch=8 wav2vec2

* Epoch 1/100


  0%|          | 0/689 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Loss: 1.22 #nan->0: 100%|██████████| 689/689 [08:54<00:00,  1.29it/s]


loss: 1.236221
acc: 0.461901
start validation


Loss: 1.14 #nan->0: 100%|██████████| 172/172 [00:50<00:00,  3.43it/s]


validation loss: 1.1562299728393555
validation accuracy: 0.5174418604651163
Save new model!
--------------------
* Epoch 2/100


...

Loss: 0.92 #nan->0: 100%|██████████| 689/689 [08:58<00:00,  1.28it/s]


loss: 0.624951
acc: 0.766328
start validation


Loss: 1.86 #nan->0: 100%|██████████| 172/172 [00:50<00:00,  3.44it/s]


validation loss: 0.814736008644104
validation accuracy: 0.6984011627906976
--------------------
* Epoch 23/100


Loss: 0.78 #nan->0:  33%|███▎      | 226/689 [02:57<06:03,  1.27it/s]

In [ ]:
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Wav2vec2/OK-4-class_wav2vec2_lr=1e-05_batch=8_16mhz_3sec.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=18)
#16mhz 3secondi tanh dropout=0.7 batch=8 wav2vec2

* Epoch 19/100


  0%|          | 0/689 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Loss: 0.71 #nan->0: 100%|██████████| 689/689 [08:51<00:00,  1.30it/s]


loss: 0.704895
acc: 0.731676
start validation


Loss: 0.93 #nan->0: 100%|██████████| 172/172 [00:50<00:00,  3.38it/s]


validation loss: 0.8468591570854187
validation accuracy: 0.6984011627906976
Save new model!
--------------------
* Epoch 20/100


...

validation loss: 0.8951614499092102
validation accuracy: 0.7122093023255814
--------------------
* Epoch 28/100


Loss: 0.18 #nan->0: 100%|██████████| 689/689 [08:52<00:00,  1.29it/s]


loss: 0.486307
acc: 0.826560
start validation


Loss: 1.34 #nan->0: 100%|██████████| 172/172 [00:50<00:00,  3.40it/s]


validation loss: 0.92388516664505
validation accuracy: 0.7136627906976745
--------------------


#### 4-class classification mockingjay

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

freq=16
duration=5
train_ds = AudiosDataset(train_4['filename'], train_4['label'])
test_ds = AudiosDataset(test_4['filename'], test_4['label'])

batch_size = 8
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'mockingjay').to(DEVICE)

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


Using cache found in /root/.cache/torch/hub/s3prl_cache/a2b432be9adba2cb59f5cf89a4cf84d5fff8ec3c9fe248ad53349694565ef8c9
for https://www.dropbox.com/s/zwsfa6w2iy2cc68/states-500000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


Loss: 1.53 #nan->0: 100%|██████████| 689/689 [17:50<00:00,  1.55s/it]


loss: 1.254204
acc: 0.458273
start validation


Loss: 1.05 #nan->0: 100%|██████████| 172/172 [01:32<00:00,  1.86it/s]


validation loss: 1.128117561340332
validation accuracy: 0.5174418604651163
Save new model!
--------------------
* Epoch 2/100


...

Loss: 0.51 #nan->0: 100%|██████████| 689/689 [16:52<00:00,  1.47s/it]


loss: 1.017587
acc: 0.596517
start validation


Loss: 1.03 #nan->0: 100%|██████████| 172/172 [01:31<00:00,  1.87it/s]


validation loss: 1.0024511814117432
validation accuracy: 0.596656976744186
Save new model!
--------------------
* Epoch 6/100


Loss: 0.98 #nan->0:  33%|███▎      | 224/689 [05:34<11:03,  1.43s/it]

In [ ]:
DEVICE = 'cuda'
model.load_state_dict(torch.load('/content/drive/MyDrive/Tesi/Mockingjay/4-class_mockingjay_lr=1e-05_batch=8_16mhz_5sec.bin'))
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr, last_epoch=5)
#16mhz 5secondi tanh dropout=0.7 batch=8 mockingjay


* Epoch 6/100


Loss: 0.76 #nan->0: 100%|██████████| 689/689 [16:37<00:00,  1.45s/it]


loss: 0.950930
acc: 0.630624
start validation


Loss: 1.21 #nan->0: 100%|██████████| 172/172 [01:30<00:00,  1.91it/s]


validation loss: 1.013177514076233
validation accuracy: 0.6170058139534884
Save new model!
--------------------
* Epoch 7/100


...

validation loss: 0.9705764651298523
validation accuracy: 0.6482558139534884
--------------------
* Epoch 14/100


Loss: 0.71 #nan->0: 100%|██████████| 689/689 [16:33<00:00,  1.44s/it]


loss: 0.603168
acc: 0.771589
start validation


Loss: 1.39 #nan->0: 100%|██████████| 172/172 [01:30<00:00,  1.90it/s]


validation loss: 0.9696465134620667
validation accuracy: 0.659156976744186
--------------------


---------------

In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'mockingjay')

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


Using cache found in /root/.cache/torch/hub/s3prl_cache/a2b432be9adba2cb59f5cf89a4cf84d5fff8ec3c9fe248ad53349694565ef8c9
for https://www.dropbox.com/s/zwsfa6w2iy2cc68/states-500000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config


In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-4
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


Loss: 1.30 #nan->0: 100%|██████████| 689/689 [16:02<00:00,  1.40s/it]


loss: 1.235103
acc: 0.485123
start validation


Loss: 1.19 #nan->0: 100%|██████████| 172/172 [01:23<00:00,  2.06it/s]


validation loss: 1.1668875217437744
validation accuracy: 0.5174418604651163
Save new model!
--------------------
* Epoch 2/100


...

validation loss: 1.2109291553497314
validation accuracy: 0.5174418604651163
--------------------
* Epoch 6/100


Loss: 0.62 #nan->0: 100%|██████████| 689/689 [15:13<00:00,  1.33s/it]


loss: 1.196313
acc: 0.502177
start validation


Loss: 1.16 #nan->0: 100%|██████████| 172/172 [01:22<00:00,  2.08it/s]


validation loss: 1.2033722400665283
validation accuracy: 0.5174418604651163
--------------------


-----------------

In [ ]:
DEVICE = 'cuda'
m = torch.hub.load('s3prl/s3prl', 'mockingjay').to(DEVICE)

In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =8e-6
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 5secondi tanh dropout=0.7 batch=8 aalbert

* Epoch 1/100


Loss: 1.07 #nan->0: 100%|██████████| 689/689 [15:25<00:00,  1.34s/it]


loss: 1.303631
acc: 0.446662
start validation


Loss: 1.16 #nan->0: 100%|██████████| 172/172 [01:19<00:00,  2.17it/s]


validation loss: 1.1412473917007446
validation accuracy: 0.5174418604651163
Save new model!
--------------------
* Epoch 2/100


...

Loss: 0.75 #nan->0: 100%|██████████| 689/689 [14:33<00:00,  1.27s/it]


loss: 0.711525
acc: 0.742380
start validation


Loss: 1.17 #nan->0: 100%|██████████| 172/172 [01:19<00:00,  2.16it/s]


validation loss: 1.0120882987976074
validation accuracy: 0.6489825581395349
--------------------
* Epoch 14/100


Loss: 0.75 #nan->0:  46%|████▌     | 315/689 [06:41<08:06,  1.30s/it]

#### Audio Albert 4-class classification - 4MHz

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#train da capo a 4 MHz. lr=1e-4

* Epoch 1/100


Loss: 1.30 #nan->0: 100%|██████████| 689/689 [01:53<00:00,  6.04it/s]


loss: 1.147412
acc: 0.514695
start validation


Loss: 1.07 #nan->0: 100%|██████████| 172/172 [00:12<00:00, 13.51it/s]


validation loss: 1.0755188465118408
validation accuracy: 0.561046511627907
Save new model!
--------------------
* Epoch 2/100


...

validation loss: 1.2715373039245605
validation accuracy: 0.6068313953488372
--------------------
* Epoch 8/100


Loss: 0.05 #nan->0: 100%|██████████| 689/689 [01:01<00:00, 11.17it/s]


loss: 0.299772
acc: 0.888244
start validation


Loss: 1.14 #nan->0: 100%|██████████| 172/172 [00:12<00:00, 14.21it/s]

validation loss: 1.3323147296905518
validation accuracy: 0.6344476744186046
--------------------


In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#train da capo a 4 MHz

* Epoch 1/100


Loss: 0.97 #nan->0: 100%|██████████| 689/689 [01:12<00:00,  9.45it/s]


loss: 1.208863
acc: 0.482221
start validation


Loss: 1.12 #nan->0: 100%|██████████| 172/172 [00:15<00:00, 11.15it/s]


validation loss: 1.1192036867141724
validation accuracy: 0.5174418604651163
Save new model!
--------------------
* Epoch 2/100


...

validation loss: 0.9638561010360718
validation accuracy: 0.6286337209302325
--------------------
* Epoch 15/100


Loss: 1.02 #nan->0: 100%|██████████| 689/689 [01:13<00:00,  9.38it/s]


loss: 0.755869
acc: 0.708454
start validation


Loss: 1.37 #nan->0: 100%|██████████| 172/172 [00:15<00:00, 11.10it/s]

validation loss: 0.9435508251190186
validation accuracy: 0.6497093023255814
--------------------


In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#train da capo a 3 MHz

* Epoch 1/100


Loss: 1.55 #nan->0: 100%|██████████| 689/689 [01:46<00:00,  6.45it/s]


loss: 1.201608
acc: 0.490566
start validation


Loss: 1.09 #nan->0: 100%|██████████| 172/172 [00:09<00:00, 17.39it/s]


validation loss: 1.1258857250213623
validation accuracy: 0.5174418604651163
Save new model!
--------------------
* Epoch 2/100


...

validation loss: 0.9823052883148193
validation accuracy: 0.6082848837209303
--------------------
* Epoch 16/100


Loss: 0.30 #nan->0: 100%|██████████| 689/689 [00:53<00:00, 12.79it/s]


loss: 0.729697
acc: 0.715167
start validation


Loss: 1.19 #nan->0: 100%|██████████| 172/172 [00:09<00:00, 17.47it/s]

validation loss: 0.9702529311180115
validation accuracy: 0.6206395348837209
--------------------


In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])
#

* Epoch 1/100


Loss: 1.28 #nan->13: 100%|██████████| 689/689 [47:36<00:00,  4.15s/it]


loss: 1.364231
acc: 0.504438
start validation


Loss: 1.10 #nan->2: 100%|██████████| 172/172 [12:09<00:00,  4.24s/it]


validation loss: 1.2355105876922607
validation accuracy: 0.5154411764705882
Save new model!
--------------------
* Epoch 2/100


Loss: 0.83 #nan->13: 100%|██████████| 689/689 [48:21<00:00,  4.21s/it]


loss: 1.318933
acc: 0.524778
start validation


Loss: 1.11 #nan->2: 100%|██████████| 172/172 [11:42<00:00,  4.09s/it]


validation loss: 1.2214609384536743
validation accuracy: 0.5161764705882353
Save new model!
--------------------
* Epoch 3/100


Loss: 0.77 #nan->5:  44%|████▍     | 304/689 [21:12<19:08,  2.98s/it]

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-6)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"])

* Epoch 1/100


Loss: 1.14 #nan->13: 100%|██████████| 689/689 [37:13<00:00,  3.24s/it]


loss: 1.571352
acc: 0.335059
start validation


Loss: 1.20 #nan->2: 100%|██████████| 172/172 [08:57<00:00,  3.13s/it]


validation loss: 1.3547611236572266
validation accuracy: 0.5154411764705882
Save new model!
--------------------
* Epoch 2/100


Loss: 1.39 #nan->11:  94%|█████████▍| 650/689 [34:23<02:26,  3.76s/it]

In [ ]:
# ##PRIMO TRAINING
# from tqdm import tqdm
# import math
# from torch import autograd


# def train(model, opt, scheduler, loss_fn, epochs, data_tr, data_val, max_stable=5):
#     best_val_loss = 1e9
#     counter = 0
#     nan=False
#     for epoch in range(epochs):
#         correct_predictions = 0
#         count=0
#         #tic = time()
#         print('* Epoch %d/%d' % (epoch+1, epochs))

#         avg_loss = 0
#         model.train()  # train mode
#         progress = tqdm(data_tr)
#         for batch in progress:
#             loss = 0
#             # data to device
#             X_batch, Y_batch = batch["representation"], batch["target"]
#             # print(X_batch.shape)
#             X_batch = X_batch.to(DEVICE)
#             Y_batch = Y_batch.to(DEVICE)
#             # with torch.autograd.detect_anomaly():
# #                     outputs = GB(inputs)
# #                     loss = LOSS(outputs, labels)
# # #                    loss.register_hook(lambda grad : print(grad))
# #                     loss.backward()
#             # set parameter gradients to zero
#             opt.zero_grad()
#             # forward
#             Y_pred = model(X_batch)
#             # print(X_batch)
#             # print(Y_pred)
#             # print(Y_batch)
#             # Y_pred = torch.tensor([e+1e-6 for e in Y_pred.reshape(1,16)[0].tolist()])
#             # Y_pred = Y_pred.reshape(8,2)
#             _, preds = torch.max(Y_pred, dim=1)
#             # loss = loss_fn(outputs[0], targets)
#             # loss = outputs[0]
#             # print(preds)
#             # print(Y_batch)
#             preds = preds.to(DEVICE)
#             Y_pred = Y_pred.to(DEVICE)
            

#             # print('---------')
#             # print('Y_pred')
#             # print(Y_pred)
            
#             # print('preds')
#             # print(preds)
#             # print('X_batch')
#             # print(X_batch)
#             # print('Y_batch')
#             # print(Y_batch)
#             # print('---------')

#             loss = loss_fn(Y_pred, Y_batch)# forward-pass
#             nan = math.isnan(loss)

#             if nan:
#               count=count+1
#               nan=False
#               continue
#               # print('---------')
#               # print('X_batch')
#               # print(X_batch.isnan().any())
#               # print('Y_pred')
#               # print(Y_pred)
#               # print(Y_pred.isnan().any())
              
#               # print('preds')
#               # print(preds.isnan().any())
              
#               # print('Y_batch')
#               # print(Y_batch.isnan().any())
#               # 
#               # print('---------')
#             # if math.isnan(loss):
#             #   print('ok')
#             # nan = True if loss is np.nan else False
#             # print('-------')
#             # print(loss)
#             # print('-------')
#             correct_predictions += torch.sum(preds == Y_batch)
#             progress.set_description(f'Loss: {loss:.2f} #nan->{count}')
            
#             loss.backward()  # backward-pass
#             # torch.nn.utils.clip_grad_norm_(model.parameters(), 25)
#             opt.step()  # update weights
#             if not scheduler is None:
#                 scheduler.step()
#             # calculate loss to show the user
#             avg_loss += loss / len(data_tr)
#       #  toc = time()
#         avg_acc = correct_predictions.double() / (len(train_ds)-(batch_size*count))
        
#         print('loss: %f' % avg_loss)
#         print('acc: %f' % avg_acc)
#         # show intermediate results
#         model.eval()  # testing mode
#         val_loss = 0
#         print("start validation")
#         for v_b in tqdm(data_val):
#             X_val, Y_val = v_b["representation"], v_b["target"]
#             Y_hat = model(X_val.to(DEVICE)).detach().cpu()# detach and put into cpu
#             val_loss += loss_fn(Y_hat, Y_val)
#         val_loss /= len(data_val)
#         print( f"validation loss: {val_loss}")
#         if val_loss <= best_val_loss and val_loss > 0:
#             counter = 0
#             print("Save new model!")
#             best_val_loss = val_loss
#             torch.save(model.state_dict(), '/content/drive/MyDrive/Tesi/AudioAlbert/wheeze_aalbert.h5')
#         else:
#             counter += 1
#         if counter == max_stable:
#             break